# Issue Commands to Change Settings

https://github.com/stilldavid/gopro-utils

In [1]:
import bs4

import data_io

import context

from gopro_helper import gopro
from gopro_helper import api
from gopro_helper import commands

# Camera Current Settings

In [27]:
k = gopro.GoProStatus(interval=1)

In [65]:
k.stop()

# Basic Commands

In [2]:
commands.set_mode_photo()

In [38]:
commands.set_mode_video()

In [53]:
commands.shutter_capture()

In [52]:
commands.shutter_stop()

In [22]:
%%time

urls = commands.get_data_urls()

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 129 ms


In [23]:
len(urls)

0

---

In [2]:
from progress_bar import bar

In [69]:
def local_data(path_save='./data'):
    """Return list of locally-stored data files
    """
    files = data_io.find(path_save, ['*.JPG', '*.jpg', '*.MP4', '*.mp4'])
    names = [os.path.basename(f) for f in files]
    
    return names

In [70]:
def update_local_data(path_save='./data', delete=False):

    local_names = local_data(path_save)
    data_urls = commands.get_data_urls()
    
    for url in bar(data_urls):
        name = os.path.basename(url)
        if name not in local_names:
            f = commands.download(url, path_save)
                
        if delete:
            commands.delete_file(url)

In [76]:
update_local_data(delete=True)

In [24]:
commands.shutter_capture()

In [25]:
commands.shutter_stop()

In [32]:
s['System']['System Busy']

0

In [44]:
commands.set_mode_photo()

In [46]:
import time

In [60]:
def stat_summary(info):
    t = info['System']['Current Time Msec']/1000
    b = info['System']['System Busy'] > 0
    
    msg = 'time: {:.3f}  Busy: {}'.format(t,b)
    
    return msg

In [74]:
commands.shutter_capture()

duration = 3
interval = 0.025

time_start = time.time()
time_now = time.time()

while time.time() - time_start < duration:
    
    time_pre = time.time()
    stat = gopro.status.fetch_camera_info()    
    time_now = time.time()

    delta = time_now - time_pre
    time_sleep = interval - delta
    
    print('{:.3f} {:s}'.format(time_now-time_start, stat_summary(stat)))

    if time_sleep > 0.001:
        time.sleep(time_sleep)

0.013 time: 4856.518  Busy: True
0.037 time: 4856.544  Busy: True
0.100 time: 4856.605  Busy: True
0.112 time: 4856.617  Busy: True
0.138 time: 4856.645  Busy: True
0.290 time: 4856.706  Busy: True
0.312 time: 4856.816  Busy: True
2.385 time: 4858.679  Busy: True
2.814 time: 4859.309  Busy: False
2.830 time: 4859.333  Busy: False
2.852 time: 4859.358  Busy: False
2.919 time: 4859.424  Busy: False
2.931 time: 4859.437  Busy: False
2.956 time: 4859.462  Busy: False
3.019 time: 4859.525  Busy: False


In [132]:
stat = gopro.status.fetch_camera_info(pretty=False)    

stat

OrderedDict([('Photo',
              {'Color': 'GoPro',
               'Default Photo Sub Mode': 'Single',
               'EV Comp': '0.0',
               'ISO MAX': '400',
               'ISO MIN': '100',
               'Megapixels': '12MP Linear',
               'Photo Sub Mode': 'Continuous',
               'Protune': 'ON',
               'RAW': 'OFF',
               'Sharpness': 'Low',
               'Shutter': 'Auto',
               'WDR': 'ON',
               'White Balance': '3000K'}),
             ('Setup',
              {'Audio Input': 'Standard Mic',
               'Auto Off': 'NEVER',
               'Auto-Rotation': 'Auto',
               'Beeps': 'Low',
               'Current Flat Mode': 'Photo',
               'Default Mode': 'Video',
               'GPS': 'ON',
               'LCD Brightness': '51',
               'LED': 'ON',
               'Language': 'English',
               'Quick Capture': 'OFF',
               'Screensaver': '1 MIN',
               'Secondary Stre

In [110]:
api.api_details['modes']

[{'display_name': 'Video',
  'path_segment': 'video',
  'settings': [{'display_name': 'Default Video Sub Mode',
    'id': 1,
    'options': [{'display_name': 'Video', 'value': 0},
     {'display_name': 'Time Lapse Video', 'value': 1},
     {'display_name': 'Video + Photo', 'value': 2},
     {'display_name': 'Looping', 'value': 3}],
    'path_segment': 'default_sub_mode'},
   {'display_name': 'Video Sub Mode',
    'id': 68,
    'options': [{'display_name': 'Video', 'value': 0},
     {'display_name': 'Time Lapse Video', 'value': 1},
     {'display_name': 'Video + Photo', 'value': 2},
     {'display_name': 'Looping', 'value': 3}],
    'path_segment': 'current_sub_mode'},
   {'display_name': 'Resolution',
    'id': 2,
    'options': [{'display_name': '4K', 'value': 1},
     {'display_name': '2.7K', 'value': 4},
     {'display_name': '2.7K 4:3', 'value': 6},
     {'display_name': '1440', 'value': 7},
     {'display_name': '1080', 'value': 9},
     {'display_name': '960', 'value': 10},
     

In [113]:
def _mode_details(mode):
    for info in api.api_details['modes']:
        if info['path_segment'] == mode:
            return info['settings']

def video_mode_details():
    return _mode_details('video')

def photo_mode_details():
    return _mode_details('photo')



In [114]:
info = photo_mode_details()

In [118]:
len(info)

15

In [119]:
k = 14

info[k]

{'display_name': 'Shutter',
 'id': 97,
 'options': [{'display_name': 'Auto', 'value': 0},
  {'display_name': '1/125', 'value': 1},
  {'display_name': '1/250', 'value': 2},
  {'display_name': '1/500', 'value': 3},
  {'display_name': '1/1000', 'value': 4},
  {'display_name': '1/2000', 'value': 5}],
 'path_segment': 'protune_exposure_time'}

In [134]:
url = api.tpl_setting.format(feature=16, value=0)

url

'http://10.5.5.9/gp/gpControl/setting/16/0'

In [135]:
api.get(api.url_mode_photo)
api.get(url)

{}

In [136]:
stat = gopro.status.fetch_camera_info(pretty=False)    

stat['Photo']

{'Color': 'GoPro',
 'Default Photo Sub Mode': 'Single',
 'EV Comp': '0.0',
 'ISO MAX': '400',
 'ISO MIN': '100',
 'Megapixels': '12MP Linear',
 'Photo Sub Mode': 'Continuous',
 'Protune': 'ON',
 'RAW': 'OFF',
 'Sharpness': 'Low',
 'Shutter': 'Auto',
 'WDR': 'ON',
 'White Balance': '3000K'}